## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [3]:
dir_path = "/Users/ecem/Documents/GitHub/seismocardiogram/data/diseased-dataset/Raw_Recordings"

In [4]:
x = np.load(dir_path + "/tr_x.npy", allow_pickle= True)
y = np.load(dir_path + "/tr_y.npy", allow_pickle= True)
z = np.load(dir_path + "/tr_z.npy", allow_pickle= True)

In [5]:
x.shape

(100,)

In [6]:
for i in range(len(x)):
    print(type(x[i]))
    print(type(y[i]))
    print(type(z[i]))
    print("******************")

<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trace.Trace'>
******************
<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trace.Trace'>
******************
<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trace.Trace'>
******************
<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trace.Trace'>
******************
<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trace.Trace'>
******************
<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trace.Trace'>
******************
<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trace.Trace'>
******************
<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trace.Trace'>
******************
<class 'obspy.core.trace.Trace'>
<class 'obspy.core.trac

## Load csv and create two seperate dataframe:
We are going to use heart dataframe to differentiate healty people from the people with diaseaes. Data's of healty people will be stored in the healty dataset while diseased people will be in diseased dataset.

In [7]:
heart = pd.read_excel("/Users/ecem/Documents/GitHub/seismocardiogram/data/diseased-dataset/Summary_Pub_Deidentified.xlsx")
df = heart.copy().fillna(0)

I have dropped some columns because they didnt have much information. ????????

In [8]:
df.drop(["Mitral valve Peak velocity (m/sec)",
         "Mitral valve Mean gradient(mmHg)", 
         "Echo available",
        "Date of collection",
        "Date of echo"], 
        inplace = True, axis = 1)
df.drop(df.tail(1).index,inplace=True)

## MS 

In [9]:
idx_MS_AS = np.where((df['Moderate or greater MS'] == 1.0)  
                  &(df['Moderate or greater MR'] == 0.0)  
                  & (df['Moderate or greater AR'] == 0.0)
                  &(df['Moderate or greater AS'] == 1.0)  
                  & (df['moderate or greater TR'] == 0.0))


In [10]:
idx_MS_AS

(array([79, 81, 83]),)

In [11]:
diseased_MS_AS = df.loc[idx_MS_AS]

In [12]:
idx_MS_AR = np.where((df['Moderate or greater MS'] == 1.0)  
                  &(df['Moderate or greater MR'] == 0.0)  
                  & (df['Moderate or greater AR'] == 1.0)
                  &(df['Moderate or greater AS'] == 0.0)  
                  & (df['moderate or greater TR'] == 0.0))

In [13]:
idx_MS_AR

(array([], dtype=int64),)

In [14]:
idx_MS_MR = np.where((df['Moderate or greater MS'] == 1.0)  
                  &(df['Moderate or greater MR'] == 1.0)  
                  & (df['Moderate or greater AR'] == 0.0)
                  &(df['Moderate or greater AS'] == 0.0)  
                  & (df['moderate or greater TR'] == 0.0))

In [15]:
idx_MS_MR

(array([46]),)

In [16]:
diseased_MS_MR = df.loc[idx_MS_MR]

## MR

In [17]:
idx_MR_AR = np.where((df['Moderate or greater MS'] == 0.0)  
                  &(df['Moderate or greater MR'] == 1.0)  
                  & (df['Moderate or greater AR'] == 1.0)
                  &(df['Moderate or greater AS'] == 0.0)  
                  & (df['moderate or greater TR'] == 0.0))

In [18]:
idx_MR_AR

(array([25, 36, 38, 43, 58, 64]),)

In [19]:
diseased_MR_AR = df.loc[idx_MR_AR]

In [20]:
idx_MR_AS = np.where((df['Moderate or greater MS'] == 0.0)  
                  &(df['Moderate or greater MR'] == 1.0)  
                  & (df['Moderate or greater AR'] == 0.0)
                  &(df['Moderate or greater AS'] == 1.0)  
                  & (df['moderate or greater TR'] == 0.0))

In [21]:
idx_MR_AS

(array([82, 89, 94]),)

In [22]:
diseased_MR_AS = df.loc[idx_MR_AS]

## AR

In [23]:
idx_AR_AS = np.where((df['Moderate or greater MS'] == 0.0)  
                  &(df['Moderate or greater MR'] == 0.0)  
                  & (df['Moderate or greater AR'] == 1.0)
                  &(df['Moderate or greater AS'] == 1.0)  
                  & (df['moderate or greater TR'] == 0.0))

In [24]:
idx_AR_AS

(array([4]),)

In [25]:
diseased_AR_AS = df.loc[idx_AR_AS]

Now its a time to seperate their waveform too.

These empty list will be used to store array values.

In [26]:
MS_AS_x, MS_AS_y, MS_AS_z = [],[],[]
MS_MR_x, MS_MR_y, MS_MR_z = [],[],[]

MR_AS_x, MR_AS_y, MR_AS_z = [],[],[]
MR_AR_x, MR_AR_y, MR_AR_z = [],[],[]

AR_AS_x, AR_AS_y, AR_AS_z = [],[],[]

In [27]:
for i in range(len(diseased_MS_AS['Patient ID'].index)):
    x_ = x[diseased_MS_AS['Patient ID'].index[i]]
    y_ = y[diseased_MS_AS['Patient ID'].index[i]]
    z_ = z[diseased_MS_AS['Patient ID'].index[i]]
    
    MS_AS_x.append(x_)
    MS_AS_y.append(y_) 
    MS_AS_z.append(z_) 
del x_, y_, z_

In [28]:
for i in range(len(diseased_MS_AR['Patient ID'].index)):
    x_ = x[diseased_MS_AR['Patient ID'].index[i]]
    y_ = y[diseased_MS_AR['Patient ID'].index[i]]
    z_ = z[diseased_MS_AR['Patient ID'].index[i]]
    
    MS_AR_x.append(x_)
    MS_AR_y.append(y_) 
    MS_AR_z.append(z_) 
del x_, y_, z_

NameError: name 'diseased_MS_AR' is not defined

In [29]:
for i in range(len(diseased_MS_MR['Patient ID'].index)):
    x_ = x[diseased_MS_MR['Patient ID'].index]
    y_ = y[diseased_MS_MR['Patient ID'].index]
    z_ = z[diseased_MS_MR['Patient ID'].index]
    
    MS_MR_x.append(x_)
    MS_MR_y.append(y_) 
    MS_MR_z.append(z_) 
del x_, y_, z_

In [30]:
print(MS_MR_x)
print(MS_MR_y)
print(MS_MR_z)

[array([<obspy.core.trace.Trace object at 0x117fda760>], dtype=object)]
[array([<obspy.core.trace.Trace object at 0x117fecf40>], dtype=object)]
[array([<obspy.core.trace.Trace object at 0x12600d250>], dtype=object)]


In [32]:
for i in range(len(diseased_MR_AS['Patient ID'].index)):
    x_ = x[diseased_MR_AS['Patient ID'].index[i]]
    y_ = y[diseased_MR_AS['Patient ID'].index[i]]
    z_ = z[diseased_MR_AS['Patient ID'].index[i]]
    
    MR_AS_x.append(x_)
    MR_AS_y.append(y_) 
    MR_AS_z.append(z_) 
del x_, y_, z_

In [33]:
for i in range(len(diseased_MR_AR['Patient ID'].index)):
    x_ = x[diseased_MR_AR['Patient ID'].index[i]]
    y_ = y[diseased_MR_AR['Patient ID'].index[i]]
    z_ = z[diseased_MR_AR['Patient ID'].index[i]]
    
    MR_AR_x.append(x_)
    MR_AR_y.append(y_) 
    MR_AR_z.append(z_) 
del x_, y_, z_

In [34]:
x[diseased_AR_AS['Patient ID'].index]

array([<obspy.core.trace.Trace object at 0x103f5bbb0>], dtype=object)

In [35]:
AR_AS_x.append(x[diseased_AR_AS['Patient ID'].index])
AR_AS_y.append(y[diseased_AR_AS['Patient ID'].index]) 
AR_AS_z.append(z[diseased_AR_AS['Patient ID'].index]) 


In [36]:
stacked_MS_AS = np.stack((MS_AS_x, MS_AS_y, MS_AS_z), axis = -1)
stacked_MS_MR = np.hstack((MS_MR_x ,MS_MR_y, MS_MR_z))

stacked_MR_AS = np.stack((MR_AS_x, MR_AS_y, MR_AS_z), axis = -1)
stacked_MR_AR = np.stack((MR_AR_x, MR_AR_y, MR_AR_z), axis = -1)

stacked_AR_AS = np.hstack((AR_AS_x, AR_AS_y, AR_AS_z))

In [37]:
print(stacked_MS_AS.shape)
print(stacked_MS_MR.shape)
print(stacked_MR_AS.shape)
print(stacked_MR_AR.shape)
print(stacked_AR_AS.shape)

(3, 3)
(1, 3)
(3, 3)
(6, 3)
(1, 3)


In [38]:
stacked_MS_MR[0][1].stats

         network: 
         station: 
        location: 
         channel: 
       starttime: 1970-01-01T00:00:00.000000Z
         endtime: 1970-01-01T00:07:01.007812Z
   sampling_rate: 256.0
           delta: 0.00390625
            npts: 107779
           calib: 1.0
      processing: ["ObsPy 1.3.0: detrend(options={}::type='linear')", "ObsPy 1.3.0: taper(max_length=None::max_percentage=0.05::side='both'::type='hann')"]

In [39]:
np.save(dir_path + "/MS_AS", stacked_MS_AS, allow_pickle= True)
np.save(dir_path + "/MS_MR", stacked_MS_MR, allow_pickle= True)

np.save(dir_path + "/MR_AS", stacked_MR_AS, allow_pickle= True)
np.save(dir_path + "/MR_AR", stacked_MR_AS, allow_pickle= True)

np.save(dir_path + "/AR_AS", stacked_AR_AS, allow_pickle= True)
